In [1]:
# Dependencies
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

### Extract JSON into DataFrames

In [2]:
# base url
base_url = "https://data.cdc.gov/api/views/r8kw-7aab/rows.json"

# run a request using our params dictionary
response = requests.get(base_url)

In [3]:
# convert response to json
cdc_data = response.json()

# Print the json (pretty printed)
print(json.dumps(cdc_data, indent=4, sort_keys=True))

{
    "data": [
        [
            "row-j96u-7nap_tvmg",
            "00000000-0000-0000-095A-9C694532D513",
            0,
            1590156655,
            null,
            1590156655,
            null,
            "{ }",
            "2020-05-22T00:00:00",
            "2020-02-01T00:00:00",
            "2020-02-01T00:00:00",
            "By week",
            "United States",
            "Week-ending",
            "0",
            "57584",
            "0.97",
            "3713",
            "0",
            "475",
            "4188",
            null
        ],
        [
            "row-3psm-6n53.mkbn",
            "00000000-0000-0000-75ED-98D3E32708F2",
            0,
            1590156655,
            null,
            1590156655,
            null,
            "{ }",
            "2020-05-22T00:00:00",
            "2020-02-08T00:00:00",
            "2020-02-08T00:00:00",
            "By week",
            "United States",
            "Week-ending",
            "1",
         

In [4]:
print(cdc_data["data"][0])

['row-j96u-7nap_tvmg', '00000000-0000-0000-095A-9C694532D513', 0, 1590156655, None, 1590156655, None, '{ }', '2020-05-22T00:00:00', '2020-02-01T00:00:00', '2020-02-01T00:00:00', 'By week', 'United States', 'Week-ending', '0', '57584', '0.97', '3713', '0', '475', '4188', None]


### Transform CDC Data into DataFrame

In [5]:
# Creating an empty Dataframe with column names only
cdc_data_df = pd.DataFrame(columns=['state','start_week', 'COVID_deaths', 'pneumonia_deaths','pneumonia_and_COVID_deaths',
                                    'influenza_deaths', 'pneumonia_influenzaor_COVID_deaths', 'total_deaths'])

In [6]:
# use iterrows to iterate through pandas dataframe
index = 0
for row in cdc_data["data"]:
    row = row
    try:
        cdc_data_df.loc[index, 'start_week'] = row[9]
        cdc_data_df.loc[index, 'state'] = row[12]
        cdc_data_df.loc[index, 'COVID_deaths'] = row[14]
        cdc_data_df.loc[index, 'pneumonia_deaths'] = row[17]
        cdc_data_df.loc[index, 'pneumonia_and_COVID_deaths'] =row[18]
        cdc_data_df.loc[index, 'influenza_deaths'] = row[19]
        cdc_data_df.loc[index, 'pneumonia_influenzaor_COVID_deaths'] = row[20]
        cdc_data_df.loc[index, 'total_deaths'] = row[15]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    index = index +1

In [7]:
cdc_data_df['start_week'] = pd.to_datetime(cdc_data_df['start_week']).dt.strftime('%m/%d/%Y')

In [8]:
# Removed Null values
cdc_data_df = cdc_data_df.fillna(0)
cdc_data_df

,state,start_week,COVID_deaths,pneumonia_deaths,pneumonia_and_COVID_deaths,influenza_deaths,pneumonia_influenzaor_COVID_deaths,total_deaths
0,United States,02/01/2020,0,3713,0,475,4188,57584
1,United States,02/08/2020,1,3715,0,507,4223,58245
2,United States,02/15/2020,0,3747,0,541,4288,57585
3,United States,02/22/2020,2,3610,0,553,4165,57640
4,United States,02/29/2020,5,3727,3,629,4358,57956
...,...,...,...,...,...,...,...,...
859,Puerto Rico,04/18/2020,18,59,0,0,71,451
860,Puerto Rico,04/25/2020,17,55,0,0,67,385
861,Puerto Rico,05/02/2020,13,30,0,0,41,224
862,Puerto Rico,05/09/2020,0,15,0,0,16,95


### Cleaning the CDC Data to megre with John Hopkins Data

In [9]:
# deleting row 0 to 16 as they are for united states as a whole
cdc_data_df = cdc_data_df[16:]
cdc_data_df

,state,start_week,COVID_deaths,pneumonia_deaths,pneumonia_and_COVID_deaths,influenza_deaths,pneumonia_influenzaor_COVID_deaths,total_deaths
16,Alabama,02/01/2020,0,55,0,14,69,1019
17,Alabama,02/08/2020,0,61,0,10,71,1107
18,Alabama,02/15/2020,0,75,0,0,81,1079
19,Alabama,02/22/2020,0,67,0,0,71,1080
20,Alabama,02/29/2020,0,63,0,14,77,1142
...,...,...,...,...,...,...,...,...
859,Puerto Rico,04/18/2020,18,59,0,0,71,451
860,Puerto Rico,04/25/2020,17,55,0,0,67,385
861,Puerto Rico,05/02/2020,13,30,0,0,41,224
862,Puerto Rico,05/09/2020,0,15,0,0,16,95


In [10]:
cdc_data_df = cdc_data_df.astype({'COVID_deaths': 'int64'})
cdc_data_df = cdc_data_df.astype({'pneumonia_deaths':'int64'})
cdc_data_df = cdc_data_df.astype({'pneumonia_and_COVID_deaths': 'int64'})
cdc_data_df = cdc_data_df.astype({'influenza_deaths':'int64'})
cdc_data_df = cdc_data_df.astype({'pneumonia_influenzaor_COVID_deaths': 'int64'})
cdc_data_df = cdc_data_df.astype({'total_deaths':'int64'})

In [11]:
cdc_data_df.dtypes

state                                 object
start_week                            object
COVID_deaths                           int64
pneumonia_deaths                       int64
pneumonia_and_COVID_deaths             int64
influenza_deaths                       int64
pneumonia_influenzaor_COVID_deaths     int64
total_deaths                           int64
dtype: object

In [12]:
#selecting only data from 04/25/2020 to 05/20/2020
cdc_may_data = cdc_data_df.loc[cdc_data_df["start_week"] >= "04/25/2020"]
cdc_may_data

,state,start_week,COVID_deaths,pneumonia_deaths,pneumonia_and_COVID_deaths,influenza_deaths,pneumonia_influenzaor_COVID_deaths,total_deaths
28,Alabama,04/25/2020,69,69,17,0,122,1009
29,Alabama,05/02/2020,61,70,16,0,116,951
30,Alabama,05/09/2020,46,59,15,0,90,728
31,Alabama,05/16/2020,14,12,0,0,22,302
44,Alaska,04/25/2020,0,0,0,0,0,48
...,...,...,...,...,...,...,...,...
847,Wyoming,05/16/2020,0,0,0,0,0,37
860,Puerto Rico,04/25/2020,17,55,0,0,67,385
861,Puerto Rico,05/02/2020,13,30,0,0,41,224
862,Puerto Rico,05/09/2020,0,15,0,0,16,95


In [13]:
cdc_may_group_data = cdc_may_data.groupby(['state']).sum()
cdc_may_group_data

,COVID_deaths,pneumonia_deaths,pneumonia_and_COVID_deaths,influenza_deaths,pneumonia_influenzaor_COVID_deaths,total_deaths
state,,,,,,
Alabama,190,210,48,0,350,2990
Alaska,0,0,0,0,0,159
Arizona,258,334,114,0,474,4165
Arkansas,43,122,0,0,159,2037
California,1179,1680,661,0,2200,17288
Colorado,488,376,237,0,627,3156
Connecticut,99,15,15,0,99,99
Delaware,144,72,54,0,162,527
District of Columbia,104,118,104,0,118,362


In [14]:
# Save Data to csv
cdc_may_group_data.to_csv("cdc_may_group_data.csv")

### Create database connection

In [15]:
connection_string = "postgres:postgres@localhost:5432/cdc_may_data_db"
engine = create_engine(f'postgresql://{connection_string}')

In [23]:
# Confirm tables
engine.table_names()

['cdc_covid_data', 'john_hopkins_data', 'combine_cdc_hopkins_data']

### Load DataFrames into database

In [17]:
cdc_may_group_data.to_sql(name='cdc_covid_data', con=engine, if_exists='append', index=True)

### Backup and Restored John Hopkins Data

#### Since we are having issues with backup and restore. Importing directly the CSV file of the team member

In [18]:
john_hopkins_file = "criselda_ETL/raw_data/john_hopkins.csv"
john_hopkins_df = pd.read_csv(john_hopkins_file)
john_hopkins_df.head()

,state,Confirmed,Deaths,Recovered,Active,People_Tested,People_Hospitalized
0,Alabama,268365.0,10469.0,0.0,257896.0,3271388.0,33973.0
1,Alaska,11345.0,294.0,8568.0,4545.0,726773.0,261.0
2,American Samoa,0.0,0.0,0.0,0.0,1873.0,0.0
3,Arizona,296916.0,13583.0,66156.0,230418.0,3143680.0,40766.0
4,Arkansas,111560.0,2345.0,67785.0,51013.0,1792689.0,12537.0


In [19]:
john_hopkins_df.to_sql(name='john_hopkins_data', con=engine, if_exists='append', index=True)

In [20]:
combined_data = pd.merge(john_hopkins_df,cdc_may_group_data, on='state')
combined_data.set_index("state", inplace=True)
combined_data

,Confirmed,Deaths,Recovered,Active,People_Tested,People_Hospitalized,COVID_deaths,pneumonia_deaths,pneumonia_and_COVID_deaths,influenza_deaths,pneumonia_influenzaor_COVID_deaths,total_deaths
state,,,,,,,,,,,,
Alabama,268365.0,10469.0,0.0,257896.0,3271388.0,33973.0,190,210,48,0,350,2990
Alaska,11345.0,294.0,8568.0,4545.0,726773.0,261.0,0,0,0,0,0,159
Arizona,296916.0,13583.0,66156.0,230418.0,3143680.0,40766.0,258,334,114,0,474,4165
Arkansas,111560.0,2345.0,67785.0,51013.0,1792689.0,12537.0,43,122,0,0,159,2037
California,1833427.0,73742.0,0.0,1759685.0,25160433.0,39006.0,1179,1680,661,0,2200,17288
Colorado,529952.0,27484.0,73634.0,439770.0,2703381.0,93679.0,488,376,237,0,627,3156
Connecticut,940155.0,79582.0,107815.0,752758.0,3589061.0,203891.0,99,15,15,0,99,99
Delaware,171738.0,5875.0,61578.0,112275.0,853420.0,2297.0,144,72,54,0,162,527
District of Columbia,163811.0,8224.0,24496.0,137589.0,789052.0,3216.0,104,118,104,0,118,362


In [21]:
# Save Data to csv
combined_data.to_csv("combined_data.csv")

In [22]:
combined_data.to_sql(name='combine_cdc_hopkins_data', con=engine, if_exists='append', index=True)